In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install gdown

In [5]:
!gdown --id 11EQcClmtteji5TIrD-oHowxcD9K8NGyx
!gdown --id 1PtC-WATPtlpkgO2uY8PyZbZCGPBaPcQD

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1PtC-WATPtlpkgO2uY8PyZbZCGPBaPcQD
From (redirected): https://drive.google.com/uc?id=1PtC-WATPtlpkgO2uY8PyZbZCGPBaPcQD&confirm=t&uuid=4fb7c491-4fec-463d-914f-70a9ecdbecc0
To: /kaggle/working/train_part2_lexrank.csv
100%|███████████████████████████████████████| 1.23G/1.23G [00:05<00:00, 222MB/s]


In [9]:
import pandas as pd

# Load the CSVs
part1 = pd.read_csv("train_part1_lexrank.csv")
part2 = pd.read_csv("train_part2_lexrank.csv")

# Combine them into a single DataFrame
train_df = pd.concat([part1, part2]).reset_index(drop=True)

print(f"Combined DataFrame shape: {train_df.shape}")
train_df.head()

Combined DataFrame shape: (117108, 3)


,article,abstract,extractive_summary
0,a recent systematic analysis showed that in 20...,background : the present study was carried out...,"therefore , the present study determines the e..."
1,it occurs in more than 50% of patients and may...,backgroundanemia in patients with cancer who a...,patients received the study treatment for 12 w...
2,"tardive dystonia ( td ) , a rarer side effect ...",tardive dystonia ( td ) is a serious side effe...,"as neck dystonia increased , the patient had a..."
3,"lepidoptera include agricultural pests that , ...",many lepidopteran insects are agricultural pes...,although immune pathways can be generally and ...
4,syncope is caused by transient diffuse cerebra...,we present an unusual case of recurrent cough ...,"cough syncope , a rare form of syncope , may b..."


In [10]:
from transformers import AutoTokenizer

# Initialize the tokenizer
model_checkpoint = "t5-small"  # or t5-base / t5-large if needed
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Ensure padding is handled correctly
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [13]:
from datasets import Dataset

# Convert DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)

# (Optional) Inspect column names if needed
# print(train_dataset.column_names)

def preprocess_function(examples):
    # Add the T5 "summarize: " prefix to the input text
    inputs = ["summarize: " + doc for doc in examples["extractive_summary"]]
    targets = examples["abstract"]
    
    # Tokenize the inputs (truncated to T5's max length)
    model_inputs = tokenizer(
        inputs, 
        max_length=512, 
        padding="max_length", 
        truncation=True
    )
    
    # Tokenize the labels/targets (use reasonable max length)
    labels = tokenizer(
        targets, 
        max_length=209,  # Abstracts are often shorter than inputs
        padding="max_length", 
        truncation=True
    )
    
    # Replace padding token id's in the labels with -100
    labels["input_ids"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in label_ids]
        for label_ids in labels["input_ids"]
    ]
    
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs


In [ ]:
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_train_dataset.save_to_disk("t5_tokenized_train_dataset")
# Check one sample
tokenized_train_dataset[0]

In [15]:
import torch
from datasets import load_dataset
from transformers import (
    T5Tokenizer, 
    T5ForConditionalGeneration, 
    TrainingArguments, 
    Trainer, 
    DataCollatorForSeq2Seq
)

In [16]:
# Load T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print("Using device:", device)

# Data collator for batch processing
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Using device: cuda


In [19]:
training_args = TrainingArguments(
    output_dir="/kaggle/working/",
    evaluation_strategy="no",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=3e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    optim="adafactor",
    fp16=True,
    push_to_hub=False,
    dataloader_num_workers=4,
    report_to="none"
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    data_collator=data_collator
)

# Train the model
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
500,3.452800
1000,3.212500
1500,3.151000
2000,3.111000
2500,3.093200
3000,3.045300
3500,3.029000
4000,3.007300
4500,2.994100
5000,2.985400


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=73195, training_loss=2.797616304264294, metrics={'train_runtime': 24770.8139, 'train_samples_per_second': 23.638, 'train_steps_per_second': 2.955, 'total_flos': 7.924803843391488e+16, 'train_loss': 2.797616304264294, 'epoch': 5.0})